# VBM661 - Veri Görselleştirilmesi - 2024-2 Son Sınav Raporu

*   Mert Çalışkan
*   N22233552

## Giriş

Kasırgalar, her yıl ABD'de milyarlarca dolarlık mülk zararına ve önemli can kaybına neden olan büyük doğal afetlerdir. Bu çalışma, 1950'den 2022'ye kadar ABD'de meydana gelen kasırgaları incelerken, veri görselleştirme tekniklerini kullanarak mevsimsel, yıllık ve coğrafi eğilimleri ortaya koymayı amaçlamaktadır. Elde edilen bulgular, afet yönetimi ve acil müdahale stratejilerinin geliştirilmesine önemli katkılar sunmayı hedeflemektedir.

### Veri Seti

Analizimiz için kullanılan veri seti, ABD Ulusal Okyanus ve Atmosfer İdaresi (NOAA) tarafından sağlanan Ulusal Hava Durumu Servisi Fırtına Tahmin Merkezi Şiddetli Hava Durumu Haritaları, Grafikleri ve Veri Sayfası’nı ve 68,693 kayıttan oluşan ['tornados'](https://github.com/rfordatascience/tidytuesday/blob/master/data/2023/2023-05-16/readme.md) isimli dosyayı temel alır. Bu veri seti, kasırgaların tarih, şiddet, izlediği yol ve neden olduğu hasar gibi 27 farklı özelliği içermektedir. Çalışma sırasında kullanılan ana özellikler aşağıda açıklanmıştır.

*   'yr' - integer - Sıralı - Kasırganın meydana geldiği yıl, 1950'den 2022'ye kadar.
*   'st' - integer - Kategorik - Eyaletin iki harflik posta kısatlması (NY = New York, NJ = New Jersey gibi…).
*   'fat' - integer - Nicel - Kasırgadan meydana gelen ölüm sayısı.

### Kod

Verilerin işlenmesi için Python programlama dili kullanılmış ve pandas, numpy, plotly gibi kütüphanelerle Google Colab ortamında veri ön işleme, analiz ve görselleştirme işlemleri gerçekleştirilmiştir. Veri setinde eksik ve tutarsız veriler incelenmiş, gerekiyorsa temizlenmiş ve analize uygun hale getirilmiştir.

## Veri Okuma

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression

# Veri setini URL'den yükleme
tornados_url = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2023/2023-05-16/tornados.csv'
df = pd.read_csv(tornados_url)


## Veri Ön İşleme ve Analizi

In [2]:
# Temel istatistikler
print(df.head())
print(df.info())
print(df.describe())

# Eksik veri kontrolü
print(df.isnull().sum())


    om    yr  mo  dy        date      time               tz  \
0  192  1950  10   1  1950-10-01  21:00:00  America/Chicago   
1  193  1950  10   9  1950-10-09  02:15:00  America/Chicago   
2  195  1950  11  20  1950-11-20  02:20:00  America/Chicago   
3  196  1950  11  20  1950-11-20  04:00:00  America/Chicago   
4  197  1950  11  20  1950-11-20  07:30:00  America/Chicago   

           datetime_utc  st  stf  ...   elon   len  wid  ns  sn   f1  f2  f3  \
0  1950-10-02T03:00:00Z  OK   40  ... -102.3  15.8   10   1   1   25   0   0   
1  1950-10-09T08:15:00Z  NC   37  ...    0.0   2.0  880   1   1   47   0   0   
2  1950-11-20T08:20:00Z  KY   21  ...    0.0   0.1   10   1   1  177   0   0   
3  1950-11-20T10:00:00Z  KY   21  ...    0.0   0.1   10   1   1  209   0   0   
4  1950-11-20T13:30:00Z  MS   28  ...    0.0   2.0   37   1   1  101   0   0   

   f4     fc  
0   0  False  
1   0  False  
2   0  False  
3   0  False  
4   0  False  

[5 rows x 27 columns]
<class 'pandas.core.frame.D

## Soru 1

### Soru 1. 1950'den 2022'ye kadar ABD'deki yıllık kasırga sayıları nasıl değişmiştir ve bu süre zarfında en fazla ve en az kasırga sayıları hangi yıllarda gözlenmiştir?

In [3]:
# Yıl bazında gruplandır ve kasırga sayılarını hesapla
grouped = df.groupby('yr').size().reset_index(name='tornado_count')

# Maksimum ve minimum kasırga sayılarını bul
max_tornado = grouped.loc[grouped['tornado_count'].idxmax()]
min_tornado = grouped.loc[grouped['tornado_count'].idxmin()]

# Eğilim için model oluştur
X = grouped['yr'].values.reshape(-1, 1)
y = grouped['tornado_count'].values
model = LinearRegression().fit(X, y)
trendline = model.predict(X).round().astype(int)

# Dinamik çizgi grafiği oluştur
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=grouped['yr'],
    y=grouped['tornado_count'],
    mode='lines',
    line=dict(color='red', width=2),
    name='Yıllara Göre<br>Kasırga Sayısı',
    hovertemplate='<b>Yıl:</b> %{x}<br><b>Kasırga Sayısı:</b> %{y}<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=grouped['yr'],
    y=trendline,
    mode='lines',
    line=dict(color='yellow', dash='dash'),
    name='Eğilim Çizgisi',
    hoverinfo='none'
))

# En fazla ve en az kasırga sayılarını etiketle
fig.add_trace(go.Scatter(
    x=[max_tornado['yr'], min_tornado['yr']],
    y=[max_tornado['tornado_count'], min_tornado['tornado_count']],
    mode='markers+text',
    marker=dict(color='red', size=6, symbol='circle', line=dict(color='blue', width=2)),
    text=['En Fazla Kasırga Sayısı', 'En Az Kasırga Sayısı'],
    textposition=['middle left', 'bottom right'],
    textfont=dict(color='white'),
    hoverinfo='text',
    hovertext=[
        f"<b>Yıl:</b> {max_tornado['yr']}<br><b>Kasırga Sayısı:</b> {int(max_tornado['tornado_count'])}",
        f"<b>Yıl:</b> {min_tornado['yr']}<br><b>Kasırga Sayısı:</b> {int(min_tornado['tornado_count'])}"
    ],
    showlegend=True,
    name='Özel Noktalar'
))

fig.update_layout(
    title="1950-2022 Yılları Arasında ABD'de Gözlemlenen Yıllık Kasırga Sayıları: En Fazla ve En Az Kasırgaların Meydana Geldiği Yıllar ve Eğilim Analizi",
    xaxis_title='Yıl',
    yaxis_title='Kasırga Sayısı',
    xaxis=dict(
        range=[1949, 2023],
        tickmode='auto',
        tickformat="d",
        showgrid=False
    ),
    yaxis=dict(
        range=[0, 1920],
        tickmode='auto',
        tickformat="d",
        showgrid=False
    ),
    template='plotly_dark',
    legend=dict(
        x=0.98, y=0.05,
        xanchor='right', yanchor='bottom',
        bgcolor='rgba(0,0,0,0)'
    ),
    autosize=True,
    dragmode=False,
    font=dict(family="Arial", size=12, color="white")
)

# Zoom ve pan özelliklerini devre dışı bırak
fig.update_geos(
    visible=False
)

fig.show(config=dict(displayModeBar=False, doubleClick=False))


### Şekil 1. Çizgi grafiği, 1950-2022 yılları arasında ABD'de kaydedilen yıllık kasırga sayılarını göstermektedir. Kasırga aktivitesindeki değişimler, afet yönetimi ve politika geliştirme süreçlerinde kritik bilgiler sunar ve bu sayede daha etkili karar verme mekanizmalarının oluşturulmasına katkıda bulunur.

### Bulgular:
* Sarı kesikli eğilim çizgisi, 1950'den 2022'ye kadar ABD'de gözlemlenen kasırga sayılarında genel bir artış eğilimini göstermektedir.
* Analiz edilen dönem boyunca kasırga sayılarında belirgin dalgalanmalar gözlemlenmiştir. 1950'lerden 1960'lara kadar olan süreçte kasırga sayısında gözlenen nispeten düşük artışlar, bu yıllarda meteorolojik şartların daha stabil olduğuna işaret edebilir. Ancak, 2000'li yılların başlangıcından itibaren yaşanan ani artışlar ve düşüşler, bu dönemdeki iklim değişikliklerinin etkisinin ve kasırga aktivitelerinin daha değişken olduğunu gösterir. Bu, afet hazırlık planlamalarında dikkate alınması gereken önemli bir faktördür.
* 2004 yılı, incelenen dönem içinde 1817 kasırga ile en yoğun yıl olarak kayıtlara geçmiştir. En düşük kasırga sayısı ise 1950 yılında, sadece 201 olay ile kaydedilmiştir. Bu, yıllık kasırga aktivitesindeki önemli değişimleri gösteren kritik noktalardır.

### Kullanılan görsel kanallar:
* Veri işareti: Çizgi grafiği, yıllara göre kasırga sayısının değişimini göstermek için kullanılmıştır.
* X-Ekseni: Yıllar, 1950'den 2022'ye Kadar (Sıralı)
* Y-Ekseni: Kasırga Sayısı (Nicel)
* Etkileşim: Grafik üzerine fare ile gelindiğinde, belirli bir yıla ait kasırga sayısı detayları görülebilmektedir.
* Etiket: "En Fazla Kasırga Sayısı" ve "En Az Kasırga Sayısı" gibi önemli veri noktaları, kullanıcının dikkatini çekmek ve bilgiyi hızla aktarmak için belirgin şekilde işaretlenmiştir.
* Sembol: Özel veri noktaları (En fazla ve en az kasırga sayısı kayıtları), mavi çerçeveli kırmızı noktalar ile daha da belirginleştirilmiştir.
* Renk ve Çizgi Stili: Ana veri serisi kırmızı renkte belirtilmiş, eğilim çizgisi ise sarı kesikli çizgiyle gösterilmiştir.


## Soru 2

### Soru 2. 1950'den 2022'ye kadar ABD'nin farklı eyaletlerinde meydana gelen kasırgalardan kaynaklanan toplam ölüm sayıları nasıl dağılmıştır?

In [4]:
# Eyalet kısaltmalarını tam isimlere çeviren sözlük oluştur
state_names = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts',
    'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
    'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
    'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming',
    'DC': 'District of Columbia', 'PR': 'Puerto Rico', 'VI': 'Virgin Islands'
}

# Eyalet bazında ölüm sayılarını hesapla
state_fatalities = df.groupby('st')['fat'].sum().reset_index()

# Eyalet isimlerini yeni bir sütun olarak ekle
state_fatalities['state_name'] = state_fatalities['st'].map(state_names)

# Plotly ile choropleth harita oluştur
fig = px.choropleth(state_fatalities,
                    locations='st',
                    locationmode='USA-states',
                    color='fat',
                    hover_name='state_name',
                    hover_data={'st': False, 'fat': True, 'state_name': False},
                    color_continuous_scale='Reds',
                    scope='usa',
                    title="1950-2022 Yılları Arasında ABD'nin Eyaletlerine Göre Kasırgalardan Kaynaklanan Toplam Ölüm Sayıları",
                    labels={'fat': 'Toplam<br>Ölüm<br>Sayısı', 'st': 'Eyalet'},
                    template='plotly_dark')  # Koyu tema ekleniyor

# Hover data formatını ayarla
fig.update_traces(
    hovertemplate='<b>Eyalet:</b> %{hovertext}<br><b>Ölüm Sayısı:</b> %{customdata[1]}<extra></extra>',
    customdata=state_fatalities[['state_name', 'fat']].values,
    hovertext=state_fatalities['state_name']
)

# Grafik ayarlarını güncelle
fig.update_layout(
    title="1950-2022 Yılları Arasında ABD'nin Eyaletlerine Göre Kasırgalardan Kaynaklanan Ölüm Sayıları",
    geo=dict(
        projection_type='albers usa',
        showframe=False,
        showcoastlines=False,
        projection_scale=1,
    ),
    margin=dict(l=0, r=0, t=75, b=25),
    autosize=True,
    dragmode=False,
    font=dict(family="Arial", size=12, color="white")
)

# Zoom ve pan özelliklerini devre dışı bırak
fig.update_geos(
    visible=False
)

fig.show(config=dict(displayModeBar=False, doubleClick=False))


### Şekil 2. Choropleth harita, 1950 ile 2022 yılları arasında ABD'nin çeşitli eyaletlerinde meydana gelen kasırgalardan kaynaklanan ölüm sayılarını renk skalası ile göstermektedir. Eyaletlerdeki ölüm sayıları, renk tonlarının koyuluğu ile ifade edilmiş, en yüksek ölüm oranlarına sahip bölgeler daha koyu renklerle belirtilmiştir. Bu görsel, özellikle yüksek risk altındaki eyaletlerde kasırga önlemleri ve hazırlıklarını yoğunlaştırmanın önemini vurgulamaktadır.

### Bulgular:
* Choropleth haritası, özellikle "Tornado Alley" olarak bilinen ve Alabama, Texas, Mississippi, Oklahoma, Tennessee, Arkansas ve Missouri eyaletlerini kapsayan [1] merkezi ABD bölgesinde yüksek ölüm oranlarını göstermektedir. Bu eyaletler, haritada koyu kırmızı renklerle belirtilerek, kasırgaların yoğun yaşandığı ve yüksek can kayıpları ile sonuçlandığı alanlar olarak vurgulanmıştır. Özellikle, Alabama 668 ölüm vakası ile en yüksek can kaybına sahip eyalet olarak dikkat çekmektedir.
* Orta Batı ve Güney eyaletleri (örneğin, Missouri, Tennessee, Kentucky) gibi bölgeler nispeten yüksek ölüm sayılarına sahipken, Batı ve Kuzeydoğu eyaletleri daha düşük ölüm oranlarına sahiptir. Bu durum, haritada daha açık renk tonları ile gösterilerek, kasırga hazırlık ve önlemlerinin bölgesel farklılıklarını ortaya koymaktadır.
* Elde edilen bulgular, kasırga yönetimi ve acil durum müdahale stratejilerinin, özellikle yüksek risk taşıyan bölgelerde güçlendirilmesi gerektiğini göstermektedir. Bu, kasırgaların sıkça rastlandığı "Tornado Alley" gibi bölgelerde, can kaybını en aza indirmek için daha etkili önlemler ve hazırlıkların yapılmasını gerektirmektedir.

### Kullanılan görsel kanallar:
* Veri işareti: Choropleth harita, eyaletler bazında ölüm sayılarını görselleştirmek için kullanılmıştır.
* X-Ekseni: Enlem (Nicel - Görselde etiketleri gösterilmemiştir)
* Y-Ekseni: Boylam (Nicel - Görselde etiketleri gösterilmemiştir)
* Çizgi: Eyalet sınırları net bir şekilde çizilmiş, böylece kullanıcılar farklı bölgeler arasındaki farklılıkları kolayca ayırt edebilir.
* Etkileşim: Harita üzerine fare ile gelindiğinde, belirli bir eyaletin ismi ve ölüm sayısı görüntülenebilmektedir.
* Renk: Ölüm sayıları, renk yoğunluğu ile ifade edilmiştir; yüksek sayılar koyu kırmızı, düşük sayılar açık tonlarda gösterilmiştir. (Nicel)




## Soru 3

### Soru 3. 1950'den 2022'ye kadar ABD'de eyalet bazlı toplam kasırga sayıları zaman içinde nasıl değişmiştir ve bu süre zarfında hangi eyaletler en çok veya en az kasırgaya maruz kalmıştır?

In [5]:
# Eyaletlere ve yıllara göre tornado sayılarını hesapla ve kısaltmaları tam isimlerle değiştir
state_year_counts = df.groupby(['yr', 'st']).size().reset_index(name='count')
state_year_counts['st'] = state_year_counts['st'].apply(lambda x: state_names[x])

# Toplam kasırga sayılarını hesaplamak için pivot tablosu oluştur
state_year_pivot = state_year_counts.pivot_table(index='st', columns='yr', values='count', fill_value=0)
state_year_cumsum = state_year_pivot.cumsum(axis=1).reset_index().melt(id_vars=['st'], var_name='yr', value_name='count')

# Son yıla göre sıralama belirle ve DataFrame'i bu sıralamaya göre düzenle
final_year_ranking = state_year_cumsum[state_year_cumsum['yr'] == state_year_cumsum['yr'].max()].groupby('st')['count'].sum().sort_values(ascending=False).index.tolist()
state_year_cumsum['fixed_rank'] = state_year_cumsum['st'].map({name: i for i, name in enumerate(final_year_ranking)})
state_year_cumsum.sort_values(['yr', 'fixed_rank'], inplace=True)

# Maksimum toplam sayıyı hesapla ve x ekseni aralığını belirle
max_cumulative_count = state_year_cumsum['count'].max()
range_x = [0, max_cumulative_count * 1.1]

# Yıllık toplam kasırga sayısını hesapla ve eyaletlere göre sırala
state_year_total = state_year_cumsum.groupby(['yr', 'st'])['count'].sum().reset_index()
state_year_total['rank'] = state_year_total.groupby('yr')['count'].rank(ascending=False, method='first')

# Eyaletlerin kasırga sayılarına göre sıralanmasını sağlayacak şekilde veriyi birleştir
merged = state_year_cumsum.merge(state_year_total[['yr', 'st', 'rank']], on=['yr', 'st'])
merged = merged.sort_values(['yr', 'rank'])
merged['yr'] = merged['yr'].astype(int)

# Eyalet sıralamasını son yıla göre sabit tut
final_year_ranking.reverse()  # Listeyi tersine çevir
merged['fixed_rank'] = merged['st'].map({name: i for i, name in enumerate(final_year_ranking)})
merged.sort_values(['yr', 'fixed_rank'], inplace=True)

fig = px.bar(merged,
             x='count',
             y='st',
             animation_frame='yr',
             orientation='h',
             title='1950-2022 Yılları Arasında Eyaletlere Göre Toplam Kasırga Sayıları',
             labels={'count': 'Toplam Kasırga Sayısı', 'st': 'Eyaletler', 'yr': 'Yıl'},
             hover_data={'count': True, 'yr': True, 'st': False},
             range_x=[0, max_cumulative_count * 1.1],
             color_discrete_sequence=['#FFD700'])

# Y ekseni sıralamasını sabitle (en yüksekten en düşüğe)
fig.update_yaxes(categoryorder='array', categoryarray=final_year_ranking)

fig.update_layout(
    template='plotly_dark',
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {
                    'frame': {'duration': 1600, 'redraw': True},
                    'fromcurrent': True,
                    'transition': {'duration': 500, 'easing': 'linear'}
                }],
                'label': 'Oynat',
                'method': 'animate'
            },
            {
                'args': [[None], {
                    'frame': {'duration': 0, 'redraw': True},
                    'mode': 'immediate',
                    'transition': {'duration': 0}
                }],
                'label': 'Duraklat',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.025,
        'xanchor': 'right',
        'y': 0.05,
        'yanchor': 'top'
    }],
    sliders=[{
        'currentvalue': {
            'prefix': 'Yıl: ',
            'visible': True,
            'xanchor': 'right'
        },
        'steps': [{'args': [[str(year)], {'frame': {'duration': 300, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300}}],
                   'label': str(year), 'method': 'animate'} for year in merged['yr'].unique()],
        'x': 0.025,
        'len': 0.9,
        'pad': {'b': 10, 't': 30}
    }],
    height=1100,
    margin=dict(l=170, r=50, t=80, b=20),
    font=dict(family="Arial", size=12, color="white")
)

# X ekseni etiketlerini özel olarak ayarla
tickvals = [i for i in range(0, int(max_cumulative_count) + 100, 1000)]
ticktext = [f'{i:,}'.replace(',', '.') for i in tickvals]
fig.update_xaxes(tickvals=tickvals, ticktext=ticktext)
#fig.update_yaxes(autorange=True)

# Çubuk kalınlıklarını artırma ve renkleri ayarla
fig.update_traces(marker=dict(line=dict(width=1), opacity=0.7),
                  selector=dict(type='bar'))

fig.show()


### Şekil 3. Bu animasyonlu yatay çubuk grafiği, 1950'den 2022'ye kadar ABD'nin her eyaletinde meydana gelen toplam kasırga sayılarını ve yıllık değişimleri göstermektedir. Her çubuk, bir eyaletin karşılaştığı toplam kasırga sayısını temsil etmekte ve animasyon, zaman içindeki değişimi vurgulamaktadır. Bu görselleştirme, kasırga etkinliğinin coğrafi dağılımını ve zaman içindeki trendleri anlamamıza yardımcı olur. Y-ekseni 2022 yılına göre en fazla toplam kasırga sayısı görülen eyaletten en aza doğru sıralanmıştır. Böylece yıllar içerisindeki değişimler, sıçramalar gözlemlenebilir bir hale gelmiştir.

### Bulgular:
* Amerika Birleşik Devletleri'nde kasırgaların en yoğun yaşandığı bölgelerden biri olan Tornado Alley, özellikle Texas, Oklahoma, Kansas ve Nebraska gibi eyaletleri kapsar. Bölgesel iklim koşullarının kasırga oluşumuna elverişli olması nedeniyle bu bölge, uzun süredir kasırga aktivitelerinin odak noktasıdır [1]. Görsel verilere göre, 1950'den 2022'ye kadar Texas, Oklahoma ve Kansas sürekli olarak yüksek kasırga sayılarıyla öne çıkmıştır.
* Tornado Alley içerisinde yer almasına rağmen Nebraska, özellikle 1974'ten sonra kasırga sayısında Florida tarafından geçilmiştir. Başlangıçta daha az kasırga etkisinde olan Florida, zamanla ani artışlar göstererek 2022 itibarıyla en yüksek toplam kasırga sayısına ulaşan dördüncü eyalet olmuştur.
* Yaklaşık 1960'tan itibaren kasırga sıklığı sıralamasında liderliği elinde tutan Texas, 2022 yılında toplam 9.265 kasırga ile rekor kırmıştır. Kansas ise, 4.429 kasırga ile Texas'ın hemen ardından gelmektedir. Bu durum, Texas'ın kasırga frekansının Kansas'a kıyasla iki katından fazla olduğunu göstermektedir.
* 1970'te Missouri, ilerleyen yıllarda kendisinden daha yüksek kasırga sayılarına ulaşacak olan Iowa, Illinois, Mississippi ve Alabama gibi eyaletlere göre nispeten yüksek bir kasırga sayısına sahiptir. Ancak, sonraki yıllarda bu eyaletler kasırga sayısında Missouri'yi geçmişlerdir. Bu durum 1970 yılından sonra Missouri'nin, bahsedilen diğer eyaletlere göre daha az kasırga frekansına maruz kaldığını açıklamaktadır.
* 2000 yılına kadar Michigan eyaleti, Tennessee, Ohio, South Carolina ve Kentucky eyaletlerine göre daha fazla kasırgaya maruz kalmıştır, ancak 2022'ye kadar bu eyaletler Michigan'ı geride bırakmıştır.
* 1993'te South Carolina, Minnesota ve Arkansas'a göre daha fazla kasırga görmüşken, 2022'ye doğru bu eyaletlerin kasırga sayıları South Carolina'yı geçmiştir.
* 1998'de Alabama eyaleti, Missouri, Colorado ve Louisiana'dan daha az kasırga sayısına sahipken, 2022 yılına kadar artan kasırga sayıları ile bu eyaletleri geride bırakmıştır.
* Rhode Island, Alaska, District of Columbia ve Virgin Islands, 1950'den 2022'ye kadar en düşük kasırga aktivitesine sahip eyaletler olmuştur.
* Bu gözlemler, özellikle Tornado Alley gibi yüksek riskli bölgelerde kasırga yönetimi ve hazırlık çalışmalarının yoğunlaştırılmasının gerekliliğini vurgulamaktadır. Ayrıca, kasırgaların az görüldüğü bölgelerde de daima hazırlıklı olunması gerektiğini hatırlatmaktadır. Bölgeler arası farklılıklar dikkate alınarak, yerel düzeyde özelleştirilmiş afet hazırlık stratejileri geliştirilmesi önerilmektedir.

### Kullanılan görsel kanallar:
* Veri işareti: Yatay çubuk grafiği, eyaletlere göre toplam kasırga sayılarını görselleştirmekte kullanılmıştır.
* X-Ekseni: Toplam Kasırga Sayısı (Nicel)
* Y-Ekseni: Eyaletler (Kategorik)
* Etkileşim: Animasyondaki her bir adım, bir takvim yılını temsil eder, böylece kullanıcılar zaman içindeki ilerlemeyi takip edebilirler.
* Etkileşim: Kullanıcı, fare ile üzerine gelindiğinde her eyalet için yıl ve toplam kasırga sayısını görebilir.


### Sonuçlar

Bu çalışmada, 1950'den 2022'ye kadar ABD'deki kasırga olaylarının yıllık ve coğrafi analizleri yapılmıştır. Çalışmamız, kasırga sayılarında zaman içinde artış eğilimi olduğunu ortaya koymuştur. Bu artış, iklim değişiklikleri ve çevresel faktörlerle bağlantılı olabilir ve daha detaylı iklim çalışmalarıyla desteklenmesi gerekmektedir.

Ölüm oranlarına yönelik analizler, özellikle Tornado Alley olarak bilinen bölgede yüksek ölüm sayılarına işaret etmekte; bu durum, bölgesel afet yönetim stratejilerinin ve acil müdahale kapasitesinin güçlendirilmesinin önemini vurgulamaktadır. Eyaletler arası farklılıklar, hazırlık ve müdahale planlamalarında bölgesel ihtiyaçları dikkate almanın gerekliliğini göstermektedir.

Eyaletlere göre toplam kasırga sayıları üzerine yapılan analizler, bazı eyaletlerin, özellikle Texas ve Oklahoma'nın uzun yıllar boyunca en çok kasırgaya maruz kalan bölgeler olduğunu belirlemiştir. Bu bilgiler, bu eyaletlerde kasırga risk yönetimi ve halkın bilinçlendirilmesi için özel stratejiler geliştirilmesi gerektiğini ortaya koymaktadır. Ayrıca, Rhode Island, Alaska, District of Columbia ve Virgin Islands, 1950'den 2022'ye kadar en düşük kasırga aktivitesine sahip eyaletler olmuştur. Kasırgaların az görüldüğü bölgelerde de daima hazırlıklı olunması gerekmektedir.

Sonuç olarak, bu çalışma, kasırga verilerinin derinlemesine analizini sağlayarak, afet yönetimi ve politika geliştirme süreçlerine katkıda bulunmuş ve kasırga hazırlık çalışmalarının etkinliğini artırma potansiyeline sahip bulgular sunmuştur.

## Kaynakça
[1] - Encyclopædia Britannica, Inc. "Tornado Alley." Britannica, t.y., www.britannica.com/place/Tornado-Alley.